## Use this to make adversarial plots

In [1]:
import matplotlib as mpl
mpl.use('Agg')

import torchvision
import torchvision.models as models
import torchvision.transforms as transforms
import torch
import torch.nn as nn
import torch.optim as optim

from sklearn.metrics import accuracy_score, classification_report

import sys, os, glob
import time
import operator
import itertools
import joblib
import numpy as np
from matplotlib import pyplot as plt
import foolbox
import getopt
sys.path.insert(0, "../util")

In [2]:
import model

%run adversarial.ipynb

## Load other helper functions and classes
from pytorch_data_loader import PytorchLoader
import helper as hp
from data_loader import UTKFace, Adience, CIFAR10
from adversarial import Attack, AttackV2

2.4.0
2.4.0


In [3]:
PHASES = ['train', 'test']
batch_size = 500
learning_rate = 0.0005
aggregate_coeff = 5

In [39]:
dataset = 'utkface_race'
gpu = 0
model_name = 'squeezenet'
epochs = ['best']
taus = [None]
alphas = [None]
with_regularization=False
sigmoid_approx=False
probabilities=True
paper_friendly_plots=True

COLORS = ['blue', 'red', 'green', 'orange', 'magenta', 'yellow', 'black', 'grey', 'cyan']

In [40]:
device = torch.device('cuda:{}'.format(gpu))

attack_names = ['DeepFool', 'CarliniWagner']

ds_obj, datasets, data_loaders = \
    hp.get_data_loder_objects(dataset, PHASES, **hp.get_loader_kwargs(batch_size))

In [41]:
def load_adversarial_objects(folder, epoch, ds_obj, device):
    object_paths = glob.glob("{}/*_epoch_{}*".format(folder, epoch))
    objects = []
    # predicted = []
    image_ids = []
    for obj in object_paths:
        image_ids.append(int(obj.split('/')[-1].rstrip('.pkl').split('_')[0]))
        adv_obj = joblib.load(obj)
        # tiled_mean = np.tile(
        #     ds_obj.data_transform.transforms[-1].mean, (adv_obj.image.shape[1], adv_obj.image.shape[2], 1)).T
        # tiled_std = np.tile(
        #     ds_obj.data_transform.transforms[-1].std, (adv_obj.image.shape[1], adv_obj.image.shape[2], 1)).T
        # processed_image = torch.tensor((adv_obj.image - tiled_mean)/tiled_std)
        # model_op = model.model_ft(processed_image.view((1,) + processed_image.size()).to(device))
        # _, preds = torch.max(model_op, 1)
        # predicted.append(preds.cpu().numpy()[0])
        objects.append(adv_obj)
    return np.array(image_ids), objects

In [42]:
def image_differences(adv_image_ids, all_adv_images, sensitive_attr, ds_obj):
    """
    Give sensitive_attr as all 1 to get all image differences in minority_differences
    Conversely, give sensitive_attr as all 0 to get all image differences in minority_differences
    """
    minority_differences, majority_differences = [], []
    for idx, img_id in enumerate(adv_image_ids):
        processed_img = ds_obj.get_image('test', int(img_id))
        raw_img = hp.inverse_transpose_images(processed_img.numpy(), ds_obj.data_transform)
        adv_img = np.moveaxis(all_adv_images[idx], 0, -1) # channels first, non normalized
        if sensitive_attr[idx] == 1:
            minority_differences.append(np.linalg.norm(raw_img - adv_img))
        else:
            majority_differences.append(np.linalg.norm(raw_img - adv_img))
            
    return minority_differences, majority_differences

In [43]:
def image_differences_individual(adv_image_ids, all_adv_images, ds_obj):
    differences = []
    for idx, img_id in enumerate(adv_image_ids):
        processed_img = ds_obj.get_image('test', int(img_id))
        raw_img = hp.inverse_transpose_images(processed_img.numpy(), ds_obj.data_transform)
        adv_img = np.moveaxis(all_adv_images[idx], 0, -1) # channels first, non normalized
        differences.append(np.linalg.norm(raw_img - adv_img))
            
    return differences

In [44]:
def set_paper_friendly_plots_params():
    plt.style.use('seaborn-paper')
    plt.rcParams['font.size'] = 10
    plt.rcParams['axes.labelsize'] = 22
    plt.rcParams['axes.labelweight'] = 'bold'
    plt.rcParams['axes.titlesize'] = 15
    plt.rcParams['axes.linewidth'] = 1.25
    plt.rcParams['xtick.labelsize'] = 16
    plt.rcParams['ytick.labelsize'] = 16
    plt.rcParams['legend.fontsize'] = 20
    plt.rcParams['figure.titlesize'] = 22
    plt.rcParams['lines.linewidth'] = 4.0
    plt.rcParams['grid.color'] = 'grey'
    plt.rcParams['grid.linestyle'] = '--'
    plt.rcParams['grid.linewidth'] = 0.25
    plt.rcParams['figure.dpi'] = 50
    plt.rcParams['savefig.dpi'] = 50

In [45]:
for epoch in epochs:
    for (tau_idx, tau), (alpha_idx, alpha) in itertools.product(*[enumerate(taus), enumerate(alphas)]):
        regularization_params = {'tau': tau, 'alpha': alpha, 'sigmoid_approx': sigmoid_approx, 
            'probabilities': probabilities, 'device': device}
        model_to_load = model.DNN(model_name=model_name, num_classes=ds_obj.num_classes(), 
            learning_rate=learning_rate, aggregate_coeff=aggregate_coeff,
            with_regularization=with_regularization, 
            regularization_params=regularization_params)

        # filename = '{}_{}_epoch_{}_lr_{}.pth'.format(model_to_load.model_name, model_to_load.criterion._get_name(), 
        #     epoch, learning_rate)
        # model_to_load.model_ft.load_state_dict(torch.load('../{}/model_weights/{}'.format(ds_obj.name, filename),
        #                                          map_location=device))
        # model_to_load.model_ft.eval()
        # print ('Loaded weights from: ../{}/model_weights/{}'.format(ds_obj.name, filename))

        complete_model_name = '{}_{}'.format(model_to_load.model_name, model_to_load.criterion._get_name()) \
            if not isinstance(model_to_load.criterion, nn.CrossEntropyLoss) else model_to_load.model_name

        for attack_name in attack_names:

            adv_folder = '../{}/adversarial_images/{}/{}'.format(ds_obj.name, 
                complete_model_name, attack_name)
            adv_image_ids, all_adv_objs = \
                load_adversarial_objects(folder=adv_folder, epoch=epoch, ds_obj=ds_obj, device=device)
            all_images_adversarial = np.array([x.image for x in all_adv_objs])

            print (adv_folder)
            print (len(glob.glob("{}/*_epoch_{}*".format(adv_folder, epoch))))

            if 'cifar' in ds_obj.name.lower():
                if ds_obj.name.lower() == 'cifar10':
                    sensitive_attrs, sensitive_attrs_names = [], []
                    for cname in ds_obj.classes:
                        sensitive_attrs_names.append(cname)
                        sensitive_attrs.append(np.array([1 if ds_obj.classes[ds_obj.test_labels[int(img_id)]] == \
                                                         cname else 0 for img_id in adv_image_ids]))
                else:
                    sensitive_attrs = [np.array(
                        [1 if ds_obj.classes[ds_obj.test_labels[int(img_id)]] == \
                            ds_obj.name.split('_')[-1].lower() \
                        else 0 for img_id in adv_image_ids])]
                    sensitive_attrs_names = [ds_obj.name.lower().split('_')[-1]]
            else:
                attr = ds_obj.name.lower().split('_')[-1]
                sensitive_attrs = [np.array([
                    ds_obj.get_image_protected_id_to_label(
                        ds_obj.get_image_protected_class('test', int(img_id), attr=attr), attr=attr) \
                                        for img_id in adv_image_ids])]
                sensitive_attrs_names = ['Black' if attr == 'race' else 'Female']
#                 sensitive_attrs_names = []

#             majority_differences, minority_differences = [], []
#             for sensitive_attr in sensitive_attrs:
#                 minority_difference, majority_difference = \
#                     image_differences(adv_image_ids, all_images_adversarial, sensitive_attr, ds_obj)
#                 majority_differences.append(majority_difference)
#                 minority_differences.append(minority_difference)
            
            hp.create_dir("plots/{}".format(ds_obj.name))
            hp.create_dir("plots/{}/{}".format(ds_obj.name, model_to_load.model_name))
            hp.create_dir("plots/{}/{}/{}".format(ds_obj.name, model_to_load.model_name, attack_name))
            dir_to_save = "plots/{}/{}/{}".format(ds_obj.name, model_to_load.model_name, attack_name)
            if paper_friendly_plots:
                set_paper_friendly_plots_params()
            # plotting all sens attrs on the same plot
            for sens_attr, sens_attr_name in zip(sensitive_attrs, sensitive_attrs_names):
                differences_sens_attrs = []
                labels = []
                print (np.unique(sens_attr))
                for attr_label in np.unique(sens_attr):
                    differences_sens_attrs.append(
                        image_differences_individual(adv_image_ids[sens_attr == attr_label], 
                                                     all_images_adversarial[sens_attr == attr_label], ds_obj))
#                     labels.append(ds_obj.get_image_protected_id_to_label(attr_id, attr=attr))
                    labels.append(attr_label)
                
                taus = np.linspace(np.min([np.min(x) for x in differences_sens_attrs]), 
                                   np.max([np.max(x) for x in differences_sens_attrs]), 2000)
                
                fig = plt.figure()
                if not paper_friendly_plots:
                    fig.suptitle(r'fraction $d_\theta > \tau$ for {}'.format(ds_obj.name), fontsize=20)
                ax = fig.add_subplot(111)
                ax.set_xlabel('Distance to Adv. Sample' + r' ($\tau$)')
                ax.set_ylabel(r'$ \widehat{I^\tau_s} $')
                
                for (idx, label), difference in zip(enumerate(labels), differences_sens_attrs):
                    frac_greater_than_tau = np.array([np.sum(difference > t) / len(difference) for t in taus])
                    ax.plot(taus, frac_greater_than_tau, color=COLORS[idx], label=label, alpha=0.5)
                
                if paper_friendly_plots and not 'carliniwagner' in attack_name.lower():
                    plt.legend()
                elif not paper_friendly_plots:
                    plt.legend()

                extension = 'png' if not paper_friendly_plots else 'pdf'
                filename = '{}_inv_cdf'.format(model_to_load.criterion._get_name()) \
                    if not isinstance(model_to_load.criterion, nn.CrossEntropyLoss) else \
                        'inv_cdf_{}_all_on_one'.format(sens_attr_name)
                plt.savefig('{}/{}.{}'.format(dir_to_save, filename, extension), bbox_inches='tight')
                plt.show()
                plt.close()
                
                    

            # print (minority_difference, majority_difference)

#             hp.create_dir("plots/{}".format(ds_obj.name))
#             hp.create_dir("plots/{}/{}".format(ds_obj.name, model_to_load.model_name))
#             hp.create_dir("plots/{}/{}/{}".format(ds_obj.name, model_to_load.model_name, attack_name))

#             dir_to_save = "plots/{}/{}/{}".format(ds_obj.name, model_to_load.model_name, attack_name)

#             taus = np.linspace(0.0, 0.5, 2000)
# #             taus = np.linspace(0.0, 2.0, 2000) # if 'deepfool' in attack_name.lower() else np.linspace(2.9, 3.1, 2000)

#             for minority_difference, majority_difference, sensitive_attr_name in zip(minority_differences, majority_differences, sensitive_attrs_names):
#                 frac_greater_than_tau_majority = np.array([np.sum(majority_difference > t) / len(majority_difference) for t in taus])
#                 frac_greater_than_tau_minority = np.array([np.sum(minority_difference > t) / len(minority_difference) for t in taus])

#                 if paper_friendly_plots:
#                     set_paper_friendly_plots_params()

#                 fig = plt.figure()
#                 if not paper_friendly_plots:
#                     fig.suptitle(r'fraction $d_\theta > \tau$ for {}'.format(ds_obj.name), fontsize=20)
#                 ax = fig.add_subplot(111)
#                 ax.plot(taus, frac_greater_than_tau_majority, color='blue', label='Other Classes')
#                 ax.plot(taus, frac_greater_than_tau_minority, color='red', label='{}'.format(sensitive_attr_name))
#                 ax.set_xlabel('Distance to Adv. Sample' + r' ($\tau$)')
#                 ax.set_ylabel(r'$ \widehat{I^\tau_s} $')
#                 plt.legend()

#                 extension = 'png' if not paper_friendly_plots else 'pdf'
#                 filename = '{}_inv_cdf'.format(model_to_load.criterion._get_name()) \
#                     if not isinstance(model_to_load.criterion, nn.CrossEntropyLoss) else \
#                         'inv_cdf_{}'.format(sensitive_attr_name)
#                 plt.savefig('{}/{}.{}'.format(dir_to_save, filename, extension), bbox_inches='tight')
#                 plt.show()
#                 plt.close()

../utkface_race/adversarial_images/squeezenet/DeepFool
3160
['asian' 'black' 'indian' 'other' 'white']
../utkface_race/adversarial_images/squeezenet/CarliniWagner
3160
['asian' 'black' 'indian' 'other' 'white']
